# References

https://pythonhosted.org/scikit-fuzzy/auto_examples/plot_defuzzify.html

https://demonstrations.wolfram.com/TriangleAreaBisectors/

https://stackoverflow.com/questions/19141432/python-numpy-machine-epsilon

https://scikit-learn.org/stable/modules/model_evaluation.html

https://www.geeksforgeeks.org/data-normalization-with-pandas/

https://neelshelar.com/explain-anfis-architecture-with-a-neat-diagram/

https://www.ques10.com/p/13355/explain-anfis-architecture-with-neat-diagram-1/

https://www.tandfonline.com/doi/pdf/10.1623/hysj.51.4.588

https://github.com/VManuelSM/Membership-functions/blob/master/membershipFunctions.py

https://functionbay.com/documentation/onlinehelp/default.htm#!Documents/fuzzymembershipfunctions.htm

https://github.com/gabrielegilardi/ANFIS

https://pyswarms.readthedocs.io/en/latest/examples/usecases/train_neural_network.html

https://towardsdatascience.com/how-to-split-a-dataframe-into-train-and-test-set-with-python-eaa1630ca7b3

In [56]:
# !pip install pyswarms
# !pip install scikit-fuzzy

# Imports

In [57]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split
import pyswarms as ps

%matplotlib inline

# Implementation

In [58]:
# one hot encoder for data in classification task and extract all classes
def build_class_matrix(y):
    """
    Builds the output array <y_out> for a classification problem. Array <y> has
    dimensions (n_samples, 1) and <y_out> has dimension (n_samples, n_classes).
    y_out[i,j] = 1 specifies that the i-th sample belongs to the j-th class.
    """
    n_samples = y.shape[0]

    # Classes and corresponding number
    y_u, idx = np.unique(y, return_inverse=True)
    n_classes = len(y_u)

    # Build the array actually used for classification
    y_out = np.zeros((n_samples, n_classes))
    y_out[np.arange(n_samples), idx] = 1.0

    return y_out, y_u


# Sigmoid activation - not used in the code (it is commented - but if we perform a sigmoid activation on result of regreesion tasks, metrics will be improved. But it is because of random p_q_r parameters and network will perform well if it is optimized.)
def sigmoid(x):
    eps = np.finfo(np.float32).eps
    if x>0:
        return 1 / (1+ np.exp(-x) + eps)
    else:
        return 1 / (1+ np.exp(-x) + eps)

In [65]:
class ANFIS:
    def __init__(self, network_IO_size, fuzzy_terms_per_feature, mf_mode):
        self.network_IO_size = network_IO_size
        self.fuzzy_terms_per_feature = fuzzy_terms_per_feature
        self.mf_mode = mf_mode
        self.mf_parameters = None
        self.p_q_r_values = None
        
    
    # triangle membership function
    def triangle_function(self, params, x):
        center = params[0]
        radius = params[1]
        abc = [center-radius, center, center+radius]
        # other membership functions can be used too! -> look at here too know more about them: https://pythonhosted.org/scikit-fuzzy/api/skfuzzy.membership.html
        value = fuzz.trimf(np.array([x]), abc)[0]
        return value
    
    
    # gbell membership function is preferred here because of easier random initialization (without constraint on order and relative magnitude of inputs).
    def gbell_function(self, params, x):
        a, b, c = params[0], params[1], params[2]
        value = fuzz.membership.gbellmf(np.array([x]), a, b, c)[0]
        return value
    
    # select membership function by the mf_mode parameter
    def mf_function(self, params, x):
        if self.mf_mode == "tri":
            return self.triangle_function(params, x)
        elif self.mf_mode == "gbell":
            return self.gbell_function(params, x)
        
    

    # generate membership values for all variables    
    def mf_generators(self, input_data):
        first_layer_out = np.zeros((self.fuzzy_terms_per_feature,self.network_IO_size[0]))
        
        fuzzy_terms_per_feature = self.fuzzy_terms_per_feature
        

        if fuzzy_terms_per_feature<=0:
            print("Invalid fuzzy term number (1 or more! is required!).")
            return 0

        else:
            for i in range(fuzzy_terms_per_feature):
                for j in range(len(input_data)):
                    first_layer_out[i][j] = self.mf_function(self.mf_parameters[i][j], input_data[j])
                
        return first_layer_out

    
    # 2nd layer of ANFIS: dot product on mu values
    def second_layer(self, first_layer_out):
        second_layer_out = np.zeros(self.fuzzy_terms_per_feature)
        for i in range(self.fuzzy_terms_per_feature):
            second_layer_out[i] = np.prod(first_layer_out[i])
            
        return second_layer_out
          
        
    # 3rd layer of ANFIS: normalizing previous layer values
    def third_layer(self, second_layer_out):
        third_layer_out = np.zeros(len(second_layer_out))
        
        if not(second_layer_out.any()):
            third_layer_out = [0.5 , 0.5]
        else:
            for i in range(len(third_layer_out)):
                third_layer_out[i] = (second_layer_out[i])/np.sum(second_layer_out)
            
        return third_layer_out
    
    
    # apply arithmetic operations on parameters of defuzzification layer
    def p_q_r_defuzzifier(self, input_data, p_q_r):
        return np.sum(input_data*p_q_r[:-1]) + p_q_r[-1]
            
        
    # 4th layer of ANFIS: defuzzification layer with p_q_r parameters
    def fourth_layer(self, input_data, third_layer_out):
        
        fourth_layer_out = np.zeros((self.network_IO_size[1], len(third_layer_out)))
        for i in range(self.network_IO_size[1]):
            for j in range(len(fourth_layer_out[0])):
                defuzzified_value = third_layer_out[j] * self.p_q_r_defuzzifier(input_data, self.p_q_r_values[i][j])
                fourth_layer_out[i][j] = defuzzified_value
            
        return fourth_layer_out


    # 5th layer of ANFIS: sum on previous layer outputs
    def fifth_layer(self, fourth_layer_out):
        fifth_layer_out = np.sum(fourth_layer_out,axis=-1)
        return fifth_layer_out
        
    
    # Sigmoid activation
    def sigmoid(self, z):
        """
        Numerically stable version of the sigmoid function (reference:
        http://fa.bianp.net/blog/2019/evaluate_logistic/#sec3.)
        """
        a = np.zeros_like(z)

        idx = (z >= 0.0)
        a[idx] = 1.0 / (1.0 + np.exp(-z[idx]))

        idx = np.invert(idx) # Same as idx = (z < 0.0)
        a[idx] = np.exp(z[idx]) / (1.0 + np.exp(z[idx]))

        return a


    # Linear Activation: Actually does nothing!
    def linear(self, x):
        return x
    
    
    # Perform an activation on last layer of ANFIS
    def activation_layer(self, fifth_layer_out, problem_category):
        # problem_category: C == Classification , R = Regression
        if problem_category == "C":
            sigmoid_out = self.sigmoid(fifth_layer_out)
            idx = np.argmax(sigmoid_out.reshape(-1,1))
            y_predict = np.zeros(self.network_IO_size[1])
            y_predict[idx]=1.0
            return y_predict
        
        elif problem_category == "R":
            return self.linear(fifth_layer_out)
#             return sigmoid(self.linear(fifth_layer_out))
        
      
    # forwardpass for computing output for one input value
    def forwardpass(self, input_data, problem_category):

        first_layer_out = self.mf_generators(input_data=input_data)
        second_layer_out = self.second_layer(first_layer_out)
        third_layer_out = self.third_layer(second_layer_out)
        fourth_layer_out = self.fourth_layer(input_data, third_layer_out)
        fifth_layer_out = self.fifth_layer(fourth_layer_out)
        final_output = self.activation_layer(fifth_layer_out, problem_category)
        return final_output
        
        
    # evaluate ANFIS with a dataset   
    def eval_data(self, dataset, problem_category, log=False):   
        y_true = dataset.values[:,-1]
        x = dataset.values[:,:-1]
        
        if problem_category=="C":
            y_true, _ = build_class_matrix(y_true)

        y_pred = np.zeros_like(y_true)
        
        if self.p_q_r_values is None:
            self.p_q_r_values = np.random.rand(self.network_IO_size[1], self.fuzzy_terms_per_feature, len(x[0])+1)
            # these values shoud be learned and tuned....
            
        if self.mf_parameters is None:
            if self.mf_mode == "tri":
                mf_dim = 2
            elif self.mf_mode == "gbell":
                mf_dim = 3
        
            self.mf_parameters = np.random.rand(self.fuzzy_terms_per_feature, len(x[0]), mf_dim)
            # these values shoud be learned and tuned....
        
        for i in range(len(x)):
            y_pred[i] = self.forwardpass(input_data=x[i], problem_category=problem_category)
        if problem_category=="R":
            mse =  mean_squared_error(y_true, y_pred)
            if log:
                print(f"MSE: {mse}")
            return [mse, y_pred]

        elif problem_category=="C": 
            acc =  accuracy_score(y_true, y_pred)  
            categorical_loss =  log_loss(y_true, y_pred)
            if log:
                print(f"Accuracy: {acc} ,Categorical loss: {categorical_loss}")
            return [categorical_loss, acc, y_pred]


# normalize a dataframe (Max-Min scaler)
def dataframe_normalizer(df):
    # copy the data
    df_min_max_scaled = df.copy()
    
    # apply normalization techniques
    for column in df_min_max_scaled.columns:
        df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())    
  
    return df_min_max_scaled


# calcultor network output size
def network_out_size_calcultor(problem_category, df=None):
    if problem_category=="R":
        return 1
    elif problem_category=="C": 
        y_u, idx = np.unique(df.values[:,-1], return_inverse=True)
        return len(y_u)

# Data

In [66]:
def task_type(dataset):
    if dataset[1] == "R":
        return "regreesion"
    elif dataset[1] == "C":
        return "classification"
    

def build_network_with_sample_dataset(dataset, fuzzy_terms_per_feature, mf_mode="gbell", split_ratio=0.2):
    df = pd.read_csv(f"./data/{dataset[0]}", header = None)
    task_type_short = dataset[1]
    print(f"This is a {task_type_short}({task_type(dataset)}) task")
    # we shoud re-scale if we want to see the real ranged prediction.
    df_train, df_test = train_test_split(df, test_size=split_ratio, shuffle=True)
    
    df_train = dataframe_normalizer(df_train)
    df_test = dataframe_normalizer(df_test)
    
    # df_train.describe()
    out_size = network_out_size_calcultor(problem_category=dataset[1], df=df_train)
    network=ANFIS(network_IO_size=(df_train.values[0].shape[0]-1,out_size), fuzzy_terms_per_feature=fuzzy_terms_per_feature, mf_mode=mf_mode)
    print("Performance of network with initial random parameters: ")
    print("Init train results:")
    init_train = network.eval_data(df_train, problem_category=dataset[1], log=True)
    print("Init test results:")
    init_test = network.eval_data(df_test, problem_category=dataset[1], log=True)
    return network, df_train, df_test, init_train, init_test

# PSO

In [67]:
def parameters_wrapper(network, mode, all_params_together=None):
    p_q_r_values_shape = network.p_q_r_values.shape
    mf_parameters_shape = network.mf_parameters.shape
    p_q_r_count = np.prod(p_q_r_values_shape)
    mf_count = np.prod(mf_parameters_shape)
    all_parameters_count = p_q_r_count + mf_count
    
#     print(p_q_r_values_shape)
#     print(mf_parameters_shape)
#     print(p_q_r_count)
#     print(mf_count)
#     print(all_parameters_count)
#     print(all_params_together)
#     print(all_params_together.shape)
#     print(p_q_r_flatten)
#     print(mf_flatten)
    
    if mode=="wrap":
        p_q_r_flatten = network.p_q_r_values.flatten()
        mf_flatten = network.mf_parameters.flatten()
        all_params_together = np.concatenate((p_q_r_flatten, mf_flatten), axis=0)
        return all_params_together
    
    elif mode=="unwrap":
        p_q_r = all_params_together[:p_q_r_count].reshape(p_q_r_values_shape)
        mf = all_params_together[p_q_r_count:].reshape(mf_parameters_shape)
        return p_q_r, mf
    
    elif mode=="params_count":
        return all_parameters_count
    
    
def suitable_form_of_network_as_function(params, mode=0, inference=False):
    problem_category=dataset[1]
    p_q_r, mf = parameters_wrapper(network, mode="unwrap", all_params_together=params)
    network.p_q_r_values = p_q_r
    network.mf_parameters = mf
    if mode==0:
        df_to_run = df_train
    elif mode==1:
        df_to_run = df_test

    out = network.eval_data(df_to_run, problem_category=dataset[1], log=False)
        
    if inference:
        return out
    else:
        loss = out[0]
        return loss


def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [suitable_form_of_network_as_function(x[i]) for i in range(n_particles)]
    return np.array(j)


def predict(params, df):
    
    problem_category=dataset[1]
    p_q_r, mf = parameters_wrapper(network, mode="unwrap", all_params_together=params)
    network.p_q_r_values = p_q_r
    network.mf_parameters = mf
    out = network.eval_data(df, problem_category=dataset[1], log=False)

    return out

In [68]:
# all_params_together = parameters_wrapper(network, mode="wrap")
# np.arange(0, 1, 0.03)
# suitable_form_of_network_as_function(params=all_params_together)

In [69]:
def run_PSO_on_network(network, n_particles=1, iters=10):
    dimensions_count = parameters_wrapper(network, mode="params_count")
    # print(dimensions_count)
    
    # Set-up hyperparameters
    options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
    # Call instance of PSO
    optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=dimensions_count, options=options)
    # Perform optimization
    cost, pos = optimizer.optimize(f, iters=iters)
    return cost, pos

# Alltogether

In [70]:
def reult_printer(pos, init_train, init_test, dataset=dataset):
    init_loss_train = init_train[0]
    init_loss_test = init_test[0]
    problem_category=dataset[1]
    inference = True
    
    # Check on tetrainst set
    train_out = suitable_form_of_network_as_function(pos, mode=0, inference=inference)
    train_loss = train_out[0]
    print(f"Train loss: {train_loss}, Init train loss: {init_loss_train}")

    # Check on test set
    test_out = suitable_form_of_network_as_function(pos, mode=1, inference=inference)
    test_loss = test_out[0]
    print(f"Test loss: {test_loss}, Init test loss: {init_loss_test}")
    
    if problem_category=="C":
        # Check on tetrainst set
        train_acc = train_out[1]
        print(f"Train acc: {train_acc}, Init train acc: {init_train[1]}")

        # Check on test set
        test_acc = test_out[1]
        print(f"Test acc: {test_acc}, Init test acc: {init_test[1]}")  

In [71]:
s=["stock_dataset.csv", "R"]
w=["wine_dataset.csv", "C"]
p=["pulsar_dataset.csv", "C"]
l=["plant_dataset.csv", "R"]

# Code works on all datasets.

In [86]:
dataset = s

network, df_train, df_test, init_train, init_test = build_network_with_sample_dataset(dataset=dataset, fuzzy_terms_per_feature=3, mf_mode="gbell", split_ratio=0.2)
# Don't change the names. These are global variable because of passing in a suitable form to PSO

This is a R(regreesion) task
Performance of network with initial random parameters: 
Init train results:
MSE: 1.3406646332954915
Init test results:
MSE: 1.4021133707936864


In [87]:
cost, pos = run_PSO_on_network(network, n_particles=100, iters=10)

2022-01-12 05:00:32,265 - pyswarms.single.global_best - INFO - Optimize for 10 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|████████████████████████████████████████████████████████████████████████████████████████████|10/10, best_cost=0.00703
2022-01-12 05:03:10,657 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.007034484250286584, best pos: [-0.18523599  0.39525363  0.36044681  0.64968409  0.08335652 -0.30749874
  0.04620737  0.12230104  0.65568481  0.17458391 -0.12110144  0.18570054
  0.37792509  0.07633825  0.21825258 -0.34777294  0.66558111  1.20971793
  0.43717829  0.55777829  1.10947873  0.11336077  0.37063121 -0.1081405
  0.21858808  0.0251866   0.59694481  0.32052406  1.25376305  0.20198665
  0.47344497  0.44563173  0.36493497  0.45099722  0.02210589  0.08023556
  0.44114211 -0.03350152  0.42665905  0.785043    0.56248504  0.33389951
  0.83495996  0.14155413  0.49893157  0.64485829  1.04824707 -0.00953864
 -0.22497878

In [88]:
reult_printer(pos, init_train, init_test)

Train loss: 0.007034484250286584, Init train loss: 1.3406646332954915
Test loss: 0.010499653358208613, Init test loss: 1.4021133707936864


In [90]:
# Check predictions of network for a custom dataframe
out_ = predict(pos, df_test)

In [91]:
out_
# [categorical_loss, acc, y_pred] for Classification task
# [mse, y_pred] for Regression task

[0.010499653358208613,
 array([0.42304904, 0.59229546, 0.52754335, 0.74771894, 0.62050896,
        0.49851703, 0.45749254, 0.48186433, 0.44228814, 0.7730616 ,
        0.38215479, 0.47337344, 0.53340977, 0.51905878, 0.5410792 ,
        0.46173247, 0.5084291 , 0.44848823, 0.48048714, 0.51461007,
        0.59347607, 0.6148059 , 0.50267023, 0.63318441, 0.50043389,
        0.5687001 , 0.48936193, 0.49871466, 0.45531364, 0.63142014,
        0.38588105, 0.65994067, 0.65132919, 0.66243556, 0.49879881,
        0.53410082, 0.51392695, 0.52904737, 0.49391241, 0.4742666 ,
        0.52097718, 0.57277136, 0.50959127, 0.62221771, 0.301177  ,
        0.43437074, 0.52768495, 0.52509291, 0.52720698, 0.46181462,
        0.48442287, 0.52579817, 0.58633026, 0.52430204, 0.60542231,
        0.49749015, 0.51041932, 0.56116658, 0.51166366, 0.61310072,
        0.66601084, 0.41677215, 0.51511935, 0.56077655, 0.52296668,
        0.53948317, 0.57737232, 0.68821564, 0.49502899, 0.52388567,
        0.54153319, 0.568